In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import itertools
from collections import Counter
from networkx.drawing.nx_agraph import graphviz_layout
from skbio.stats.composition import ilr
from skbio.stats.composition import clr
from skbio.stats.composition import multiplicative_replacement
import seaborn as sns
from matplotlib import rcParams
sns.set()
sns.set(font_scale=1.5)

In [2]:
# Bacerial phylum->strain lineages
BacterialLineages= pd.read_csv("./Files/BacterialInformation/myGenomes.lineage.csv")
BacTaxLineageDF = BacterialLineages.set_index('taxid').copy()
BacACCLineageDF = BacterialLineages.set_index('accession').copy()

In [4]:
BacterialLineages.head()

,accession,taxid,superkingdom,phylum,class,order,family,genus,species,strain
0,NZ_CM000441.1,455631,Bacteria,Firmicutes,Clostridia,Clostridiales,Peptostreptococcaceae,Clostridioides,Clostridioides difficile,Clostridioides difficile QCD-66c26
1,NZ_CM000714.1,526973,Bacteria,Firmicutes,Bacilli,Bacillales,Bacillaceae,Bacillus,Bacillus cereus,Bacillus cereus m1293
2,NZ_CM000742.1,526997,Bacteria,Firmicutes,Bacilli,Bacillales,Bacillaceae,Bacillus,Bacillus mycoides,Bacillus mycoides DSM 2048
3,NZ_CM000741.1,526994,Bacteria,Firmicutes,Bacilli,Bacillales,Bacillaceae,Bacillus,Bacillus cereus,Bacillus cereus AH1273
4,NC_013791.2,398511,Bacteria,Firmicutes,Bacilli,Bacillales,Bacillaceae,Bacillus,Bacillus pseudofirmus,Bacillus pseudofirmus OF4


In [5]:
import itertools
ACCdic = {}
DirtyDic = {}
accessionList = []
allStrainNames = []
accTaxDict = {"Bacterial_Strain":[], "Accession":[], "TaxID":[], "Species":[], "Genus":[], 'Family':[], 'Order':[], 'Class':[], 'Phylum':[]}
with open("./Files/BacterialInformation/fastaToName.txt") as fastName:
    for line1,line2,line3 in itertools.zip_longest(*[fastName]*3):
        if '_PLASMID' in line2:
            accTaxDict['Bacterial_Strain'].append(str(line2.split("org:")[1].replace("_PLASMID","").strip()))
            accTaxDict['Accession'].append(line2.split(">")[1].split("|")[0].strip())
            accTaxDict['TaxID'].append(line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip())
            try:
                accTaxDict['Species'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['species'])
                accTaxDict['Genus'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['genus'])
                accTaxDict['Family'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['family'])
                accTaxDict['Order'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['order'])
                accTaxDict['Class'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['class'])
                accTaxDict['Phylum'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['phylum'])
            except:
                try:
                    accTaxDict['Species'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['species'])
                    accTaxDict['Genus'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['genus'])
                    accTaxDict['Family'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['family'])
                    accTaxDict['Order'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['order'])
                    accTaxDict['Class'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['class'])
                    accTaxDict['Phylum'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['phylum'])
                except:
                    print("CANT FIND: "+str(line2.split("org:")[1].strip()))
        else:
            accTaxDict['Bacterial_Strain'].append(str(line2.split("org:")[1].strip()))
            accTaxDict['Accession'].append(line2.split(">")[1].split("|")[0].strip())
            accTaxDict['TaxID'].append(line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip())
            try:
                accTaxDict['Species'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['species'])
                accTaxDict['Genus'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['genus'])
                accTaxDict['Family'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['family'])
                accTaxDict['Order'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['order'])
                accTaxDict['Class'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['class'])
                accTaxDict['Phylum'].append(BacTaxLineageDF.loc[line2.split("tax_id:")[1].split("|")[0].split("_")[0].strip()]['phylum'])
            except:
                try:
                    accTaxDict['Species'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['species'])
                    accTaxDict['Genus'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['genus'])
                    accTaxDict['Family'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['family'])
                    accTaxDict['Order'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['order'])
                    accTaxDict['Class'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['class'])
                    accTaxDict['Phylum'].append(BacACCLineageDF.loc[line2.split(">")[1].split("|")[0].strip()]['phylum'])
                except:
                    print("CANT FIND: "+str(line2.split("org:")[1].strip()))

            
        
        if '_PLASMID' in line2:
            ACCdic[line1.split("==>")[1].split("<==")[0].split(".fasta")[0].strip()] = line2.split("org:")[1].split("_PLASMID")[0].replace("*","X").replace("#","").replace(",","").replace("'","").replace(";","").replace("(","").replace(")","").strip()
            DirtyDic[line1.split("==>")[1].split("<==")[0].split(".fasta")[0].strip()]= line2.split("org:")[1].strip()
        else:
            ACCdic[line1.split("==>")[1].split("<==")[0].split(".fasta")[0].strip()] = line2.split("org:")[1].replace(",","").replace("*","X").replace("#","").replace("'","").replace(";","").replace("(","").replace(")","").strip()
            DirtyDic[line1.split("==>")[1].split("<==")[0].split(".fasta")[0].strip()]= line2.split("org:")[1].strip()
            
del ACCdic['GCF_000162015.1_ASM16201v1_genomic']
del ACCdic['GCF_001879645.1_ASM187964v1_genomic']

In [6]:
accTaxDictDF1 = pd.DataFrame(data=accTaxDict)
accTaxDictDF1.set_index("Bacterial_Strain", inplace=True)
for row in accTaxDictDF1.index:
    try:
        accTaxDictDF1.at[row, 'Species'] = str(accTaxDictDF1.at[row, 'Species']).replace("'","").replace(" ", '_').replace("[","").replace("]","").replace(".","")
        accTaxDictDF1.at[row, 'Genus'] = str(accTaxDictDF1.at[row, 'Genus']).replace("'","").replace(" ", '_').replace("[","").replace("]","").replace(".","")
        accTaxDictDF1.at[row, 'Family'] = str(accTaxDictDF1.at[row, 'Family']).replace("'","").replace(" ", '_').replace("[","").replace("]","").replace(".","")
        accTaxDictDF1.at[row, 'Order'] = str(accTaxDictDF1.at[row, 'Order']).replace("'","").replace(" ", '_').replace("[","").replace("]","").replace(".","")
        accTaxDictDF1.at[row, 'Class'] = str(accTaxDictDF1.at[row, 'Class']).replace("'","").replace(" ", '_').replace("[","").replace("]","").replace(".","")
        accTaxDictDF1.at[row, 'Phylum'] = str(accTaxDictDF1.at[row, 'Phylum']).replace("'","").replace(" ", '_').replace("[","").replace("]","").replace(".","")
    except:
        continue
        
accTaxDictDF2 = accTaxDictDF1.reset_index().copy()
for row in accTaxDictDF2.index:
    if accTaxDictDF2.at[row,'TaxID'] == str(475375) :
        accTaxDictDF2.drop(index=row, inplace=True)
        
accTaxDictDF3 = accTaxDictDF2.set_index("Bacterial_Strain").copy()
accTaxDictDF3.head()

,Accession,TaxID,Species,Genus,Family,Order,Class,Phylum
Bacterial_Strain,,,,,,,,
Clostridioides_difficile_QCD66c26,NZ_CM000441.1,455631,Clostridioides_difficile,Clostridioides,Peptostreptococcaceae,Clostridiales,Clostridia,Firmicutes
Bacillus_cereus_m1293_strainm1293,NZ_CM000714.1,526973,Bacillus_cereus,Bacillus,Bacillaceae,Bacillales,Bacilli,Firmicutes
Bacillus_mycoides_DSM_2048_strainDSM_2048,NZ_CM000742.1,526997,Bacillus_mycoides,Bacillus,Bacillaceae,Bacillales,Bacilli,Firmicutes
Bacillus_cereus_AH1273_strainAH1273,NZ_CM000741.1,526994,Bacillus_cereus,Bacillus,Bacillaceae,Bacillales,Bacilli,Firmicutes
Bacillus_pseudofirmus_OF4_strainOF4,NC_013791.2,398511,Bacillus_pseudofirmus,Bacillus,Bacillaceae,Bacillales,Bacilli,Firmicutes


In [7]:
accTaxDictDF = accTaxDictDF3.copy()
for row in accTaxDictDF.index:
    if accTaxDictDF.at[row,'Phylum'] == 'nan' or accTaxDictDF.at[row,'Phylum'] == np.nan:
        
        accTaxDictDF.at[row,'Phylum'] = "Unclassified_Bacteria"
        accTaxDictDF.at[row,'Class'] = "Unclassified_Bacteria"
        accTaxDictDF.at[row,'Order'] = "Unclassified_Bacteria"
        accTaxDictDF.at[row,'Family'] = "Unclassified_Bacteria"
        accTaxDictDF.at[row,'Genus'] = "Unclassified_Bacteria"
        
    elif accTaxDictDF.at[row,'Class'] == 'nan' or accTaxDictDF.at[row,'Class'] == np.nan:
        
        accTaxDictDF.at[row,'Class'] = "Unclassified_"+str(accTaxDictDF.at[row,'Phylum'])
        accTaxDictDF.at[row,'Order'] = "Unclassified_"+str(accTaxDictDF.at[row,'Phylum'])
        accTaxDictDF.at[row,'Family'] = "Unclassified_"+str(accTaxDictDF.at[row,'Phylum'])
        accTaxDictDF.at[row,'Genus'] = "Unclassified_"+str(accTaxDictDF.at[row,'Phylum'])

    
    elif accTaxDictDF.at[row,'Order'] == 'nan' or accTaxDictDF.at[row,'Order'] == np.nan:
        
        accTaxDictDF.at[row,'Order'] = "Unclassified_"+str(accTaxDictDF.at[row,'Class'])
        accTaxDictDF.at[row,'Family'] = "Unclassified_"+str(accTaxDictDF.at[row,'Class'])
        accTaxDictDF.at[row,'Genus'] = "Unclassified_"+str(accTaxDictDF.at[row,'Class'])
    
    elif accTaxDictDF.at[row,'Family'] == 'nan' or accTaxDictDF.at[row,'Family'] == np.nan:
        
        accTaxDictDF.at[row,'Family'] = "Unclassified_"+str(accTaxDictDF.at[row,'Order'])
        accTaxDictDF.at[row,'Genus'] = "Unclassified_"+str(accTaxDictDF.at[row,'Order'])
        
        
    elif accTaxDictDF.at[row,'Genus'] == 'nan' or accTaxDictDF.at[row,'Genus'] == np.nan:
        
        accTaxDictDF.at[row,'Genus'] = "Unclassified_"+str(accTaxDictDF.at[row,'Family'])
        
    else:
        continue

In [8]:
#accTaxDictDF.to_csv("./Files/Dataframes/Master_Rename.csv")

## Load Small Sample Information DataFrame

In [9]:
allCohortInfoDF = pd.read_csv("./Files/Cohorts/SampleMappingInformation.csv").set_index("SampleName")
allCohortInfoDF.head()

,Cohort,Total_Reads,Mapped,Fraction_Mapped
SampleName,,,,
CSMA8M9R,nonIBD,19884444.0,9350717.0,0.470253
MSM6J2RU,nonIBD,19874995.0,10663018.0,0.536504
MSM79H6N,nonIBD,17635326.0,9182872.0,0.520709
MSM9VZNH,nonIBD,17498519.0,13125443.0,0.750089
MSM9VZF7,nonIBD,17406321.0,9450732.0,0.542948


## LOAD ORIGINAL STRAIN ABUNDANCE DF

In [10]:
def pullGoodSamples(df, cohortname, totalMapped):
    
    #Will pull only the good samples
    df2 = df.copy()
    df3 = df2.loc[~df2.index.duplicated(keep='first')].copy()
    sampleNameList = list(allCohortInfoDF[(allCohortInfoDF['Mapped']>totalMapped) & (allCohortInfoDF['Cohort'] == cohortname)].copy().index)
    df4= df3.loc[df3.index.intersection(sampleNameList)]
    df4.index.name = 'SampleName'
    
    #Will make sure that any duplicated strains are added together before continuing
    df4.rename(columns={"Faecalibacterium_prausnitzii_A2165_strainA2165":"Faecalibacterium_prausnitzii_strainA2165"}, inplace=True)
    df5 = df4.groupby(lambda x:x, axis=1).sum().copy()

    return(df5)

In [11]:
def knightsColumnRename(df):
    
    #Will pull only the good samples
    df2 = df.copy()
    sampleNames = {}    
    for sample in df2.index:
        sampleNames[sample]=sample.split("_R1")[0].strip()
    df2.rename(index=sampleNames, inplace=True)
    
    #Will make sure that any duplicated strains are added together before continuing
    df2.rename(columns={"Faecalibacterium_prausnitzii_A2165_strainA2165":"Faecalibacterium_prausnitzii_strainA2165"}, inplace=True)
    df5 = df2.groupby(lambda x:x, axis=1).sum().copy()

    return(df5)

In [12]:
crcDF1 = pd.read_csv("./Files/Dataframes/PlasmidRecalcDFs/CRC_plasmid-free.tsv", sep="\t").set_index("SampleName")
japaneseDF = pd.read_csv("./Files/Dataframes/PlasmidRecalcDFs/Japanese_plasmid-free.tsv", sep="\t").set_index("SampleName")

In [13]:
crcDF = pullGoodSamples(crcDF1, 'CRC', 250000)
japaneseDF = pullGoodSamples(japaneseDF, 'Japanese', 250000)

## Clean up strain Dataframes, Turn to species, filter

In [14]:
def filterNoise(df):
    df3 = df.copy()
    
    # Quickly check if the entire column is bad
    lowMax = []
    for column in df3.columns:
        if df3[column].max() < 0.00001:
            lowMax.append(column)
    #Drop bad columns
    df4 = df3.drop(columns=lowMax).copy()
    
    #Create new dataframe and check noise levels
    thresh=0.00001
    df5 = df4.apply(lambda x: x.where(x > thresh, 0), axis=0).copy()
                
    return(df5)

In [15]:
def strainToSpecies(df):
    cleandf = df.copy()
    
    # Convert strain names into species names
    tempDictionary = { k:v for (k,v) in zip(accTaxDictDF.index, accTaxDictDF['Species'])}
    cleandf.rename(columns=tempDictionary,inplace=True)
    
    df3 = cleandf.groupby(lambda x:x, axis=1).sum().copy()
    
    return(df3)

In [18]:
def strainToSpeciesG(df):
    cleandf = df.copy()
    
    # Convert strain names into genus names
    tempDictionary = { k:v for (k,v) in zip(accTaxDictDF.index, accTaxDictDF['Genus'])}
    cleandf.rename(columns=tempDictionary,inplace=True)
                  
    df3 = cleandf.groupby(lambda x:x, axis=1).sum().copy()
    return(df3)

In [19]:
def StrainToPhylum(df):
    cleandf = df.copy()
    
    # Convert Strain names into Phylum names
    tempDictionary = { k:v for (k,v) in zip(accTaxDictDF.index, accTaxDictDF['Phylum'])}
    cleandf.rename(columns=tempDictionary,inplace=True)
    
    
    df3 = cleandf.groupby(lambda x:x, axis=1).sum().copy()
    return(df3)

## Now Filter DataFrames

In [20]:
#CRC DATAFRAME
CRCDF1 = crcDF.copy()
CRCDF2 = filterNoise(CRCDF1)

CRCDF3 = strainToSpecies(CRCDF2)
CRCDF4 = strainToSpeciesG(CRCDF2)
crcPhylum = StrainToPhylum(CRCDF2)

In [21]:
#Japanese DATAFRAME
japaDF1 = japaneseDF.copy()
japaDF2 = filterNoise(japaDF1)

japaDF3 = strainToSpecies(japaDF2)
japaDF4 = strainToSpeciesG(japaDF2)
japanesePhylum = StrainToPhylum(japaDF2)

## Export Strain Dataframes

In [22]:
#CRC
CRCDF2.to_csv("./Files/Dataframes/strainDataframes/masterCRC_allStrains.csv")
#JAPANESE
japaDF2.to_csv("./Files/Dataframes/strainDataframes/masterJapanese_allStrains.csv")

## Export Species Dataframes

In [23]:
#CRC
CRCDF3.to_csv("./Files/Dataframes/speciesDataframes/masterCRC_allSpecies.csv")
#JAPANESE
japaDF3.to_csv("./Files/Dataframes/speciesDataframes/masterJapanese_allSpecies.csv")

## Export Genus Dataframes

In [24]:
#CRC
CRCDF4.to_csv("./Files/Dataframes/genusDataframes/masterCRC_allGenera.csv")
#JAPANESE
japaDF4.to_csv("./Files/Dataframes/genusDataframes/masterJapanese_allGenera.csv")

## Export Phylum Dataframes

In [25]:
#CRC
crcPhylum.to_csv("./Files/Dataframes/phylumDataframes/masterCRC_allPhylum.csv")
#JAPANESE
japanesePhylum.to_csv("./Files/Dataframes/phylumDataframes/masterJapanese_allPhylum.csv")

## Prevalence Filters

In [26]:
def StrainFilterForFunctional(df):
    dfo = df.copy()
    dfo.replace(0.0, np.nan, inplace=True)
    # If Strain is not in 10% of samples drop strain
    dfo.dropna(axis=1, how='any', thresh=round(len(dfo.index)*.1), inplace=True)
    dfo.fillna(0.0, inplace=True)         
    return(dfo)

In [27]:
def prevalenceFilter(df):
    dfo = df.copy()
    dfo.replace(0.0, np.nan, inplace=True)
    # Filter Species dataframe. If species is not in 90% of samples drop species
    dfo.dropna(axis=1, how='any', thresh=round(len(dfo.index)*.9), inplace=True)
    dfo.fillna(0.0, inplace=True)
    dfo2 = dfo.div(dfo.sum(axis=1), axis=0).copy()            
    return(dfo2)

In [28]:
def prevalenceFilter100(df):
    dfo = df.copy()
    dfo.replace(0.0, np.nan, inplace=True)
    # Filter Species dataframe. If species is not in all samples drop species
    dfo.dropna(axis=1, how='any', thresh=round(len(dfo.index)), inplace=True)
    dfo.fillna(0.0, inplace=True)
    dfo2 = dfo.div(dfo.sum(axis=1), axis=0).copy()            
    return(dfo2)

## Start to pull the Prevalence Groups and collapse to Species/Genus

In [30]:
################################################# STRAIN 100 ######################################################
#This will only keep strains found in every sample (100% Prevalence)
CRCStrains100 = prevalenceFilter100(CRCDF2)
JapaneseStrains100 = prevalenceFilter100(japaDF2)


#CRC
CRCStrains100.to_csv("./Files/Dataframes/strainDataframes/masterCRC_100%Prev_Strains.csv")
#JAPANESE
JapaneseStrains100.to_csv("./Files/Dataframes/strainDataframes/masterJapanese_100%Prev_Strains.csv")

In [31]:
########################################## STRAIN 90 ############################################################
#This will only keep strains found in 90% of samples (90% Prevalence)
CRCStrains90 = prevalenceFilter(CRCDF2)
JapaneseStrains90 = prevalenceFilter(japaDF2)

#CRC
CRCStrains90.to_csv("./Files/Dataframes/strainDataframes/masterCRC_90%Prev_Strains.csv")
#JAPANESE
JapaneseStrains90.to_csv("./Files/Dataframes/strainDataframes/masterJapanese_90%Prev_Strains.csv")

In [32]:
#################################################### SPECIES 100 #################################################
CRCSpecies100 = prevalenceFilter100(CRCDF3)
JapaneseSpecies100 = prevalenceFilter100(japaDF3)

#CRC
CRCSpecies100.to_csv("./Files/Dataframes/speciesDataframes/masterCRC_100%Prev_Species.csv")
#JAPANESE
JapaneseSpecies100.to_csv("./Files/Dataframes/speciesDataframes/masterJapanese_100%Prev_Species.csv")

In [33]:
#################################################### SPECIES 90 #################################################
CRCSpecies90 = prevalenceFilter(CRCDF3)
JapaneseSpecies90 = prevalenceFilter(japaDF3)


#CRC
CRCSpecies90.to_csv("./Files/Dataframes/speciesDataframes/masterCRC_90%Prev_Species.csv")
#JAPANESE
JapaneseSpecies90.to_csv("./Files/Dataframes/speciesDataframes/masterJapanese_90%Prev_Species.csv")

In [34]:
#################################################### GENERA 90 #################################################
CRCGenera90 = prevalenceFilter(CRCDF4)
JapaneseGenera90 = prevalenceFilter(japaDF4)


#CRC
CRCGenera90.to_csv("./Files/Dataframes/genusDataframes/masterCRC_90%Prev_Genera.csv")
#JAPANESE
JapaneseGenera90.to_csv("./Files/Dataframes/genusDataframes/masterJapanese_90%Prev_Genera.csv")